In [1]:
import json
import pandas as pd

pd.set_option('display.max_columns', None)

with open('source_files/assistance-listings.json') as f:
    assistanceListingsListString = json.load(f)

with open('source_files/dictionary.json') as f:
    dictionaryList = json.load(f)
    
with open('source_files/organizations.json') as f:
    organizationsList = f.read()

# load dictionary values into dataframe
for i in dictionaryList['_embedded']['jSONObjectList']:
    if i['id'] == 'assistance_type':
        allData = []
        for e in i['elements']:
            for s in e['elements']:
                row = {
                    'parent_code': e['code'],
                    'parent_description': e['description'],
                    'parent_element_id': e['element_id'], 
                    'parent_value': e['value'],
                    'code': s['code'],
                    'element_id': s['element_id'],
                    'value': s['value']
                }
                allData.append(row)
        assistanceTypesDf = pd.read_json(json.dumps(allData)).set_index('element_id')
    if i['id'] == 'applicant_types':
        applicantTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')
    if i['id'] == 'assistance_usage_types':
        assistanceUsageTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')
    if i['id'] == 'beneficiary_types':
        beneficiaryTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')


# load organization values into dataframe
organizationsDf = pd.read_json(organizationsList).set_index('orgKey')

# load assistance listing values
assistanceListingsDf = pd.json_normalize(assistanceListingsListString)

# load USASpending.gov obligations data
usaspending_df = pd.read_csv('source_files/usaspending_db_obligations_by_program.csv')

# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'parentProgramId',
    'latest',
    'revision',
    'fiscalYearLatest',
    'id',
    'data.title',
    'data.website',
    'data.financial.accounts',
    'data.financial.obligations',
    'data.financial.isFundedCurrentFY',
    'data.objective',
    'data.fiscalYear',
    'data.description',
    'data.eligibility.applicant.types',
    'data.eligibility.beneficiary.types',
    'data.eligibility.beneficiary.isSameAsApplicant',
    'data.eligibility.assistanceUsage.types',
    'data.programNumber',
    'data.organizationId',
    'data.alternativeNames',
    'data.assistanceTypes',
    'data.relatedPrograms'
}

# remove unnecessary columns
assistanceListingsDf = assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList)

/var/folders/9h/7x28ccgx1fb1751n43ffqkm00000gn/T/ipykernel_21600/1248327364.py:31: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  assistanceTypesDf = pd.read_json(json.dumps(allData)).set_index('element_id')
/var/folders/9h/7x28ccgx1fb1751n43ffqkm00000gn/T/ipykernel_21600/1248327364.py:33: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  applicantTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('element_id')
/var/folders/9h/7x28ccgx1fb1751n43ffqkm00000gn/T/ipykernel_21600/1248327364.py:35: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  assistanceUsageTypesDf = pd.read_json(json.dumps(i['elements'])).set_index('e

In [2]:
# this section could be cleaned up; originally used for setting up / loading into memory "one dataframe to rule them all"
# for data anlaysis, but some of this computation is unnecessary for the current use case of exporting data to CSV

# transform and explode rows, to build out a comprehensive and flattened data model
assistanceListingsDf = assistanceListingsDf.explode('data.eligibility.applicant.types', ignore_index=True) \
                    .explode('data.assistanceTypes', ignore_index=True) \
                    .explode('data.eligibility.beneficiary.types', ignore_index=True) \
                    .explode('data.eligibility.assistanceUsage.types', ignore_index=True) \
                    .explode('data.financial.obligations', ignore_index=True) \
                    .explode('data.financial.accounts', ignore_index=True) \
                    .explode('data.relatedPrograms', ignore_index=True)
assistanceListingsDf['data.financial.accounts'] = assistanceListingsDf['data.financial.accounts'].apply(lambda x: x.get('code') if x is not None else None)
assistanceListingsDf['data.financial.obligations.isRecoveryAct'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('isRecoveryAct') if x is not None else None)
assistanceListingsDf['data.financial.obligations.obligationId'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('obligationId') if x is not None else None)
assistanceListingsDf['data.financial.obligations.assistanceType'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('assistanceType') if x is not None else None)
assistanceListingsDf['data.financial.obligations.description'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('description') if x is not None else None)
assistanceListingsDf['data.financial.obligations.values'] = assistanceListingsDf['data.financial.obligations'].apply(lambda x: x.get('values') if x is not None else None)
assistanceListingsDf = assistanceListingsDf.explode('data.financial.obligations.values', ignore_index=True)
assistanceListingsDf['data.financial.obligations.values.year'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('year') if x is not None else None)
assistanceListingsDf['data.financial.obligations.values.estimate'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('estimate') if x is not None else 0)
assistanceListingsDf['data.financial.obligations.values.actual'] = assistanceListingsDf['data.financial.obligations.values'].apply(lambda x: x.get('actual') if x is not None else 0)

# convert datatypes
assistanceListingsDf['data.eligibility.applicant.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.applicant.types'])
assistanceListingsDf['data.financial.obligations.assistanceType'] = pd.to_numeric(assistanceListingsDf['data.financial.obligations.assistanceType'])
assistanceListingsDf['data.eligibility.assistanceUsage.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.assistanceUsage.types'])
assistanceListingsDf['data.eligibility.beneficiary.types'] = pd.to_numeric(assistanceListingsDf['data.eligibility.beneficiary.types'])
assistanceListingsDf['data.organizationId'] = pd.to_numeric(assistanceListingsDf['data.organizationId'])

# join in different datasets
assistanceListingsDf = assistanceListingsDf.join(applicantTypesDf, on='data.eligibility.applicant.types', rsuffix='_applicant')
assistanceListingsDf = assistanceListingsDf.join(assistanceTypesDf, on='data.financial.obligations.assistanceType', rsuffix='_assistanceTypes')
assistanceListingsDf = assistanceListingsDf.join(assistanceUsageTypesDf, on='data.eligibility.assistanceUsage.types', rsuffix='_assistanceUsage')
assistanceListingsDf = assistanceListingsDf.join(beneficiaryTypesDf, on='data.eligibility.beneficiary.types', rsuffix='_beneficiary')
assistanceListingsDf = assistanceListingsDf.join(organizationsDf, on='data.organizationId', rsuffix='_organization')

In [5]:
""" programs """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.title',
    'data.description',
    'data.objective',
    'data.organizationId',
    'l1ShortName',
    'l1Name',
    'l2Name'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_programs.csv', index=False)

In [ ]:
""" program_to_applicant_type """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_applicant_type.csv', index=False)

In [ ]:
""" program_to_category """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value_assistanceUsage'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_category.csv', index=False)

In [ ]:
""" program_to_beneficiary """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'value_beneficiary'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_beneficiary.csv', index=False)

In [ ]:
""" program_to_assistance_type """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.financial.obligations.assistanceType',
    'parent_code',
    'parent_description',
    'parent_element_id',
    'parent_value',
    'code_assistanceTypes',
    'value_assistanceTypes',
    'code_assistanceUsage'
}
# remove unnecessary columns and save to csv
assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList).drop_duplicates(ignore_index=True).to_csv('data_extracts/db_program_to_assistance_type.csv', index=False)

In [51]:
""" program obligations """
# designate the columns we are interested in and will maintain in the data model
allowedColumnsList = {
    'data.programNumber',
    'data.financial.obligations.obligationId',
    'data.financial.obligations.values.year',
    'data.financial.obligations.values.actual',
    'data.financial.obligations.values.estimate'
}

merge_df = usaspending_df.astype({'cfda_number': 'str', 'fiscal_year': 'float64'}) \
                         .groupby(['cfda_number', 'fiscal_year']) \
                         .agg({'sum':'sum'})
export_df = assistanceListingsDf.drop(columns=set(assistanceListingsDf)-allowedColumnsList) \
                                .reset_index() \
                                .drop_duplicates(ignore_index=True) \
                                .groupby(['data.programNumber','data.financial.obligations.values.year']) \
                                .agg({'data.financial.obligations.values.estimate':'sum', 
                                      'data.financial.obligations.values.actual':'sum'})

export_df = export_df.join(merge_df, on=['data.programNumber', 'data.financial.obligations.values.year']).reset_index()
export_df.columns = ['cfda_number', 'fiscal_year', 'sam_estimate', 'sam_actual', 'usa_spending_obligation']
export_df.to_csv('data_extracts/db_program_obligations.csv', index=False)

In [52]:
export_df.groupby('fiscal_year').sum()

,cfda_number,sam_estimate,sam_actual,usa_spending_obligation
fiscal_year,,,,
2007.0,10.86011.47712.11464.01464.01564.10064.10964.1...,4.317300e+10,1.540427e+12,0.000000e+00
2008.0,10.86011.47712.11414.91231.00764.01464.01564.1...,2.114923e+12,5.281481e+11,0.000000e+00
2009.0,10.86011.47712.11414.90114.91231.00764.01464.0...,6.042496e+12,5.398288e+09,0.000000e+00
2010.0,14.25114.90114.91231.00759.05964.20166.31270.0...,8.430279e+11,2.508904e+12,0.000000e+00
2011.0,14.25145.16959.05964.00564.02664.03564.11664.1...,3.174468e+09,5.201411e+07,0.000000e+00
2012.0,10.08911.61712.55614.25114.88015.93216.71019.0...,2.956107e+09,2.344568e+10,0.000000e+00
2013.0,10.08911.61712.55612.59914.12214.12614.13514.1...,4.576860e+10,6.807217e+10,0.000000e+00
2014.0,10.08910.58710.70011.61712.36012.40012.55612.5...,1.151700e+11,3.237201e+11,0.000000e+00
2015.0,10.09910.11710.17110.40410.40610.40710.41010.4...,6.476100e+12,5.882687e+12,0.000000e+00
